## Import libraries

In [1]:
import src.config as config

2025-06-20 14:43:47.077 | INFO     | src.config:<module>:11 - PROJ_ROOT path is: C:\Repositories\ds-lstm-ibov


## ETL — Extract, Transform, Load of Raw Dataset

This section is responsible for the Extraction phase of the ETL process, pulling historical financial data from multiple sources as defined in the configuration file [dataset.yaml](../configs/dataset.yaml).

- Data Storage: Each dataset is stored locally in the data/raw/ directory, organized and saved individually as .csv files for traceability and versioning.
- Data Cleaning:
  - Missing values are treated using a standard cleaning strategy.
  - Features with low variance (threshold < 0.01) are removed to reduce noise and improve modeling efficiency. In this execution, no low-variance columns were found.

In [2]:
from src import dataset

dataset.main(
    # ---- REPLACE DEFAULT PATHS AS APPROPRIATE ----
    asset = '^BVSP',
    asset_focus = 'Close',
    years = 1
    # -----------------------------------------
    )

2025-06-20 14:43:50.195 | INFO     | src.dataset:main:32 - Starting raw data loading...
2025-06-20 14:43:50.195 | INFO     | src.dataset:main:38 - Requesting information between 2015-06-23 and 2025-06-20
2025-06-20 14:43:50.207 | INFO     | src.utils.dataset.dataset_loading_strategy:_load_from_yfinance:56 - Downloading BOVESPA (^BVSP) from yfinance...


[*********************100%***********************]  1 of 1 completed


2025-06-20 14:43:53.638 | INFO     | src.utils.dataset.dataset_loading_strategy:_load_from_yfinance:56 - Downloading S&P500 (^GSPC) from yfinance...


[*********************100%***********************]  1 of 1 completed


2025-06-20 14:43:56.474 | INFO     | src.utils.dataset.dataset_loading_strategy:_load_from_yfinance:56 - Downloading BITCOIN (BTC-USD) from yfinance...


...

2025-06-20 14:44:04.941 | INFO     | src.utils.dataset.dataset_loading_strategy:_load_from_yfinance:56 - Downloading ACUCAR (SB=F) from yfinance...


[*********************100%***********************]  1 of 1 completed


2025-06-20 14:44:07.643 | INFO     | src.utils.dataset.dataset_loading_strategy:load:81 - Downloading SELIC (11) from the Central Bank of Brazil API...
2025-06-20 14:44:29.741 | INFO     | src.utils.dataset.dataset_loading_strategy:load:81 - Downloading CDI (12) from the Central Bank of Brazil API...
2025-06-20 14:44:32.022 | INFO     | src.utils.dataset.dataset_loading_strategy:load:81 - Downloading SELIC_Anual (1178) from the Central Bank of Brazil API...
2025-06-20 14:44:34.322 | INFO     | src.utils.dataset.dataset_loading_strategy:load:81 - Downloading SELIC_Meta_Anual (432) from the Central Bank of Brazil API...
2025-06-20 14:45:04.615 | WARNING  | src.utils.dataset.dataset_loading_strategy:_request_bcb_series:118 - Erro ao conectar à API do BCB: Expecting value: line 1 column 1 (char 0)
2025-06-20 14:45:04.615 | WARNING  | src.utils.dataset.dataset_loading_strategy:_load_single_ticker:94 - No data returned for SELIC_Meta_Anual (432)
2025-06-20 14:45:06.624 | INFO     | src.utils

INFO:src.utils.dataset.clean_strategy:Executing CleanMissingValues...
INFO:src.utils.dataset.clean_strategy:Executing CleanLowVariance with threshold=0.01...
INFO:src.utils.dataset.clean_strategy:Columns removed due to low variance: Index([], dtype='object')


2025-06-20 14:45:41.799 | SUCCESS  | src.dataset:main:64 - Raw data successfully loaded...
            ('Close', '^BVSP')  ('High', '^BVSP')  ('Low', '^BVSP')  \
2025-06-11            137128.0           137531.0          135628.0   
2025-06-12            137800.0           137931.0          136175.0   
2025-06-13            137213.0           137800.0          136586.0   
2025-06-16            139256.0           139988.0          137212.0   
2025-06-17            138840.0           139497.0          138293.0   
2025-06-18            138717.0           139161.0          138443.0   
2025-06-19            138717.0           139161.0          138443.0   

            ('Open', '^BVSP')  ('Volume', '^BVSP')  ('Close', '^GSPC')  \
2025-06-11           136443.0            8767100.0         6022.240234   
2025-06-12           137127.0            7124600.0         6045.259766   
2025-06-13           137800.0            8628300.0         5976.970215   
2025-06-16           137212.0            762

## Feature Engineering — Time Series Preparation

This part of the pipeline is responsible for transforming the cleaned dataset into structured features suitable for training time series models. It includes the following key steps:
- Feature Generation: Constructs relevant features based on historical market data.
- Dataset Splitting: The dataset is split into training and testing sets using a consistent strategy to preserve temporal structure.
- Time Series Windowing: Converts the sequential data into overlapping windows, enabling the model to learn temporal dependencies.
- Saving Artifacts: Both training and testing sets are stored for reproducibility, along with the transformation pipelines applied during preprocessing.

In [3]:
from src import features

features.main(
    # ---- REPLACE DEFAULT PATHS AS APPROPRIATE ----
    input_path = config.PROCESSED_DATA_DIR / "dataset.csv",
    train_dir = config.PROCESSED_DATA_DIR,
    test_dir = config.PROCESSED_DATA_DIR,
    # -----------------------------------------
)

2025-06-20 14:46:03.626 | INFO     | src.features:main:31 - Generating features from dataset...
2025-06-20 14:46:03.659 | INFO     | src.utils.features.splitter_strategy:split:15 - Splitting dataset into training and testing sets...
2025-06-20 14:46:03.676 | INFO     | src.utils.features.generator_strategy:generate:18 - Generating Timeseries from dataset...
2025-06-20 14:46:03.759 | SUCCESS  | src.features:main:46 - Saving train features in C:\Repositories\ds-lstm-ibov\data\processed...
2025-06-20 14:46:03.812 | SUCCESS  | src.features:main:50 - Saving test features in C:\Repositories\ds-lstm-ibov\data\processed...
2025-06-20 14:46:03.814 | SUCCESS  | src.features:main:54 - Features generation complete.
2025-06-20 14:46:03.815 | INFO     | src.features:main:56 - Saving transformers...
2025-06-20 14:46:03.816 | INFO     | src.utils.features.splitter_strategy:split:15 - Splitting dataset into training and testing sets...
2025-06-20 14:46:03.821 | INFO     | src.features:main:72 - Total t

## Modeling

The modeling process begins with loading the training dataset. Next, the base model is constructed, and both the compilation and training strategies are defined. With the pipeline structure in place, the model is trained over 100 epochs using an iterative approach to adjust the weights.

During training, key metrics are monitored, including accuracy, loss (error), validation accuracy and loss (on unseen data), as well as the learning rate.

In [4]:
from src.modeling import train

train.main(
    # ---- REPLACE DEFAULT PATHS AS APPROPRIATE ----
    X_path = config.PROCESSED_DATA_DIR / "X_train.npy",
    y_path = config.PROCESSED_DATA_DIR / "y_train.npy"
    # -----------------------------------------
)

2025-06-20 14:46:03.959 | INFO     | src.modeling.train:main:29 - Loading training dataset...
2025-06-20 14:46:03.993 | INFO     | src.modeling.train:main:37 - Building model...
2025-06-20 14:46:03.993 | INFO     | src.modeling.train:main:43 - Selecting compile strategy...
2025-06-20 14:46:03.993 | INFO     | src.modeling.train:main:46 - Selecting training strategy...
2025-06-20 14:46:03.993 | INFO     | src.modeling.train:main:49 - Building model training pipeline template...
2025-06-20 14:46:03.993 | INFO     | src.modeling.train:main:56 - Training model...
Epoch 1/1000
157/157 ━━━━━━━━━━━━━━━━━━━━ 11s 25ms/step - loss: 0.1318 - mae: 0.3082 - mse: 0.1749 - r2_score: 0.4797 - rmse: 0.3988 - smape: 84.2287 - val_loss: 0.0660 - val_mae: 0.2022 - val_mse: 0.0521 - val_r2_score: 0.2572 - val_rmse: 0.2299 - val_smape: 40.0433 - learning_rate: 0.0010
Epoch 2/1000
157/157 ━━━━━━━━━━━━━━━━━━━━ 3s 19ms/step - loss: 0.0665 - mae: 0.1733 - mse: 0.0586 - r2_score: 0.8318 - rmse: 0.2383 - smape: 5

## Predict

In the prediction stage, the trained model is loaded along with the input data. The model then performs inference, generating predictions based on the provided data.


In [6]:
from src.modeling import predict
predict.main(
    # ---- REPLACE DEFAULT PATHS AS APPROPRIATE ----
    input_path = config.PROCESSED_DATA_DIR / "dataset.csv",
    preprocessor_path = config.PROCESSED_DATA_DIR / "preprocessor.pkl",
    model_path = config.MODELS_DIR / "Sequential_epoch137_loss0.0081.keras",  # Select the best model  
    postprocessor_path = config.PROCESSED_DATA_DIR / "postprocessor.pkl",
    output_path = config.PROCESSED_DATA_DIR / "dataset_report.csv",
    # -----------------------------------------
)

2025-06-20 14:58:34.306 | INFO     | src.modeling.predict:main:28 - Performing inference for model...
2025-06-20 14:58:34.363 | INFO     | src.modeling.predict:main:32 - Input data shape: (21, 48)
2025-06-20 14:58:34.382 | INFO     | src.utils.features.generator_strategy:generate:18 - Generating Timeseries from dataset...


c:\Repositories\ds-lstm-ibov\.venv\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 678ms/step
            ('Close', '^BVSP')  ('High', '^BVSP')  ('Low', '^BVSP')  \
2025-06-19         138717.0000         139161.000     138443.000000   
2025-06-20         135588.8125         136287.875     134823.609375   

            ('Open', '^BVSP')  ('Volume', '^BVSP')       type  
2025-06-19           138844.0            8323400.0       True  
2025-06-20           135498.5           10258546.0  Predicted  
2025-06-20 14:58:35.575 | SUCCESS  | src.modeling.predict:main:75 - Inference complete.


C:\Repositories\ds-lstm-ibov\src\modeling\predict.py:61: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_report = df_report.ffill()


## Plot

Open the [Power BI Report]((../reports/pbi/amp-fynance.pbip)) and refresh data.